# 1、百度网盘AI大赛——图像处理挑战赛：手写文字擦除第2名方案


## 1.1、项目背景，赛题来源

**[百度网盘AI大赛：手写文字擦除挑战赛](https://aistudio.baidu.com/aistudio/competition/detail/129/0/introduction)**

手需要建立模型，对比赛给定的带有手写痕迹的试卷图片进行处理，擦除相关的笔，还原图片原本的样子，并提交模型输出的结果图片。
如下，上图为模型输入，下图为gt(我们期望模型的输出)。

![](https://ai-studio-static-online.cdn.bcebos.com/684eeddbce984c5da16e8f0c190970015d69777b6e244e5b99e85ab6cb249cfb) ![](https://ai-studio-static-online.cdn.bcebos.com/d9901307a8334d038ebc3e5e172c17b56c7c786f3b104b00aef1c3824453ca15)


## 1.2、本项目使用说明
1. 本项目所有代码全部透明，即函数在notebook中定义，在notebook中使用。

2. 对B榜结果进行预测请直接一键运行《加载模型进行预测B.ipynb》。

3. 公开项目提供了一个bestmodel模型以及训练日志，在model/deeplab_augument_alldata2中。

4. 重头对训练数据集进行训练，首先一键运行《解压并处理数据.ipynb》获得训练数据集；然后一键运行《使用全数据训练模型.ipynb》即可。

## 1.3、模型思路及调优

### 1.3.1[我们对赛题的认识](https://aistudio.baidu.com/paddle/forum/topic/show/992448)

> 首先，是图片尺寸极度不统一，最小500多，最大5000多，差一个数量级，因此，对所有图片采用统一的resize效果很可能不理想。

> 然后，图像大小与试卷内容相关，因此将大试卷裁剪为几个小的部分是一个可行的办法，为了保证图片尺寸统一，将图像裁剪到2048大小，使用重叠裁剪。

> 最后，手写文字擦除数据集与赛题给出的相关数据集分布完全不同，比如手写文字擦除，我们不需要做生成任务，因为擦除之后用白色填充即可。另外手写字体与印刷字体也是一大挑战。

### 1.3.2[使用基线模型发现的问题](https://aistudio.baidu.com/aistudio/projectdetail/3257671)

> 首先，效果不理想，只能得到0.55分!![](https://ai-studio-static-online.cdn.bcebos.com/02358dd393ab46f98c8aa697014c24ce0bb2b1abf67c4324a2c2c315e49b16f5)

> 然后，如上图所示(红色标注位置)，第一个是使用全卷积网络模型生成的印刷部分非常模糊，因为前处理resize和后处理resize，这就会导致模型丢分。第二个是错误的将印刷体删除，反而保留了相当多的手写字体。

> 最后，模型与手写文字不够贴合，见1.3.1我们对赛题的认识

### 1.3.3重新对赛题进行定位

> 首先，我们将赛题定位为语义分割问题，这是因为，我们不需要生成，而且生成的区域由于非常窄小，使用插值算法或的直接空白填补都是直接可用的。所以，只要我们能够提高分割精度miou就可以提高最后的score。

> 然后，由于基线模型不能区分手写与印刷，我们采用更强的mask作为模型学习的指引。也就是说，我们将印刷字体、手写字体、背景各自作为一类，这样进行语义分割时模型就必须将手写字体与印刷字体严格的区分开，而不是像基线，大范围内是手写字体的才小心翼翼地擦除。

> 最后，因为图像尺寸相差过大，我们选择重叠裁剪的方式统一图像尺寸，并扩充了图像数据集1081——>1913。

## 1.4 我们优化的路程

日期不一定准确，大概是那天

12月6号，关注到该比赛，觉得这个任务有点意思，准备参加

12月10号，查看官方发的相关论文，看了三篇，学习到不少

12月15号，查看训练数据集，发帖进行研究并与相关论文里的数据集进行对比，实际上，后续我们的工作也基本是在这个指导下进行逐步的完善的。

12月22号，跑基线模型，0.50258分

12月24号，[换另一个开源模型](https://aistudio.baidu.com/aistudio/projectdetail/3234194)，0.55269分

12月24号到1月1号开始跑相关基线模型，不知道为啥我就跑不到0.55599分。
![](https://ai-studio-static-online.cdn.bcebos.com/71079ed7d538497d95832b8ad28b4a1fe3655806d7da4b1db5a9836165dd25c3)

在这段期间，我开始思考，并分析模型的输出结果图，随着提交的次数越来越多，想要放弃的念头也就越来越重。但是，放弃是不可能的，做一件事就是做一件事。这种执着，是能让我走过过去的困难，也能让我战胜将来的困难。我心如铁，坚不可摧，最后的最大的挑战实际上不是别人，是自己的心。敢于放弃之前的成绩吗？敢于探索未知的领域吗？敢于一直保持着卷卷人的状态吗？敢于付出行动吗？最终，我们果断的舍弃掉了两个基线模型，并从0开始编写代码。

1月2号，将赛题任务定位为语义分割，并对模型进行训练，达到0.636分。

随后，在1月5号，到1月11号对图片后处理进行调优，包括腐蚀和膨胀操作的次数、系数，消除红色字体的阈值，以及mask生成的相关阈值。
![](https://ai-studio-static-online.cdn.bcebos.com/02c19650adac4e329b4ae7132d84e1f1253812b352e5496a9684c5906305b686)

再次将分数提高了一个点，到0.648分。
但是，在这个分数的，又再次卡住了，虽然通过进一步的调优操作再次提高的0.004分，但是我想提高到0.66分，因此需要迈出更大一步。
![](https://ai-studio-static-online.cdn.bcebos.com/3ec5462dcb4a44beb0a006f92c4bb9bc2ffb5af5e64146c792739d696e86dc58)

![](https://ai-studio-static-online.cdn.bcebos.com/406abb8013eb47b6865871eade71fb4ed2b9ed1df7ac4f0aa8d7f3b10b630224)

在这10几天内，我也一直在思考，思考深度学习，思考人工智能。在打比赛的过程中，[写了一篇SwinT关于让SwinTransformer快速上手的项目](https://aistudio.baidu.com/aistudio/projectdetail/3288357)，欢迎来fork。另外，也荣幸的成为了东南大学飞桨领航团的团长，[这是我们飞桨领航团的aistudio主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/1834570)，欢迎关注~

通过思考这三个问题：1、大模型剪枝(大模型压缩)，大模型剪枝就是因为训练的大模型很多连接都是0权重，曾经听李宏毅老师说过，训练大模型再剪枝而不是训练一个小模型，是因为训练小模型的效果达不到大模型剪枝的效果。2、GoogleLenet在网络层中间加入的分类监督。3、基线模型中，使用mask作为模型权重学习的指引，后续推理时，则舍弃这块。

容易得出结论：深度学习模型，很可能参数量上绝对是有能力解决这个问题的，如果让人类手动调很有可能一个小网络就能做的非常好(这太累了，hhh)。我们所作的很多工作，实际上是为了让模型更容易地优化到那个点上。包括更改模型架构，调整超参数，更改损失函数，添加引导标签。

基于此，我们将原训练deeplabv3的单损失函数替换为混合损失函数，这是最有可能直接影响到模型参数训练过程的因素，因为它直面数据；并且使用几乎所有的图片增强策略。通过这小小的改动，我们将原miou0.828分数再次提高，达到0.842。最终的得分也成功的提升到了0.66分。

![](https://ai-studio-static-online.cdn.bcebos.com/49791bc517cb4fd8b2a70787fca805965323424a6ed74b7289532a7cfe650c8d)



## 1.5关于为什么我们采用训练集数据进行验证
有以下两点理由：

1、我们不希望放弃每一个数据，之前做[中央凹定位](https://aistudio.baidu.com/aistudio/projectdetail/3168864)时采用5折交叉验证，数据稀缺时，我们总是希望用来训练模型的数据越多越好。

2、在训练数据和测试数据分布几乎一致时，训练精度与验证精度将不会相差很多。加入验证数据集可以看到miou，这方便我们挑选模型，因为我们不能根据loss去挑选模型。

## 1.6[使用SwinT搭建过SwinUnet模型](https://aistudio.baidu.com/aistudio/projectdetail/3288357)
batch_size为2时就满显存的，实在是跑不动，期待未来的硬件能再进一步发展。